In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from sklearn.metrics import accuracy_score


In [ ]:

# Налаштування параметрів
num_classes = 10
num_features = 784
learning_rate = 0.001
training_steps = 3000
batch_size = 256
display_step = 100

n_hidden_1 = 128
n_hidden_2 = 256

# Завантажуємо набір даних MNIST
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
x_train, x_test = x_train / 255., x_test / 255.

# Формуємо датасет для навчання
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

# Клас повнозв'язного шару
class DenseLayer(tf.Module):
    def __init__(self, in_features, out_features, activation=None):
        super().__init__()
        self.w = tf.Variable(tf.random.normal([in_features, out_features]), name="w")
        self.b = tf.Variable(tf.zeros([out_features]), name="b")
        self.activation = activation

    def __call__(self, x):
        y = tf.matmul(x, self.w) + self.b
        return self.activation(y) if self.activation else y

# Клас нейромережі
class NN(tf.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = DenseLayer(num_features, n_hidden_1, activation=tf.nn.sigmoid)
        self.layer2 = DenseLayer(n_hidden_1, n_hidden_2, activation=tf.nn.sigmoid)
        self.out_layer = DenseLayer(n_hidden_2, num_classes, activation=tf.nn.softmax)

    def __call__(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        return self.out_layer(x)

# Функція втрат
def cross_entropy(y_pred, y_true):
    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred), axis=1))

# Функція точності
def accuracy(y_pred, y_true):
    return accuracy_score(y_true, np.argmax(y_pred, axis=1))

# Функція навчання
def train(neural_net, input_x, output_y, optimizer):
    with tf.GradientTape() as g:
        pred = neural_net(input_x)
        loss = cross_entropy(pred, output_y)
    gradients = g.gradient(loss, [neural_net.layer1.w, neural_net.layer1.b,
                                  neural_net.layer2.w, neural_net.layer2.b,
                                  neural_net.out_layer.w, neural_net.out_layer.b])
    optimizer.apply_gradients(zip(gradients, [neural_net.layer1.w, neural_net.layer1.b,
                                              neural_net.layer2.w, neural_net.layer2.b,
                                              neural_net.out_layer.w, neural_net.out_layer.b]))
    return loss

# Функція для запуску тренування з вибором методу оптимізації
def train_network(optimizer_type):
    neural_net = NN()
    loss_history = []
    accuracy_history = []

    if optimizer_type == "SGD":
        optimizer = tf.optimizers.SGD(learning_rate)
    elif optimizer_type == "BatchGD":
        optimizer = tf.optimizers.Adam(learning_rate)

    for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
        loss = train(neural_net, batch_x, batch_y, optimizer)
        if step % display_step == 0:
            pred = neural_net(batch_x)
            acc = accuracy(pred.numpy(), batch_y.numpy())
            loss_history.append(loss.numpy())
            accuracy_history.append(acc)
            print(f"{optimizer_type} - Крок {step}, Втрата: {loss.numpy():.4f}, Точність: {acc:.4f}")

    return loss_history, accuracy_history

In [ ]:
# Запускаємо тренування для SGD та Batch Gradient Descent
loss_sgd, acc_sgd = train_network("SGD")
loss_batchgd, acc_batchgd = train_network("BatchGD")

In [ ]:
# Візуалізація результатів
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(loss_sgd, label='Втрати SGD')
plt.plot(loss_batchgd, label='Втрати BatchGD')
plt.xlabel("Кроки (x100)")
plt.ylabel("Втрати")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(acc_sgd, label='Точність SGD', color='red')
plt.plot(acc_batchgd, label='Точність BatchGD', color='blue')
plt.xlabel("Кроки (x100)")
plt.ylabel("Точність")
plt.legend()

plt.show()